# Session 1 
## Changing the solver and discretisation 

First we'll import pybamm and load the DFN model

In [1]:
import pybamm

In [3]:
model = pybamm.lithium_ion.DFN()

There are a number of different ODE and DAE solvers available in pybamm (see the documentation for an up-to-date list). Each of the solvers have a number of options, some of which are common (e.g. relative and absolute tolerances), and some of which are solver-specific (e.g. the integration "mode" of the `CasadiSolver`). Here we are going to use the `CasadiSolver` in "fast" mode, which ignores any events, and with both absolute and relative tolerances set to `1e-3`. 

We create the solver object and pass it to the simulation

In [4]:
# atol是绝对公差，rtol是相对公差
solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")
# 默认的求解器也是casadisolver
sim = pybamm.Simulation(model, solver=solver)
sim.solve([0, 3600])

We can the plot the results

In [5]:
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

除了更改求解器之外，您可能还希望更改网格中的点数。让我们看一下默认的点数，它们存储为字典，其键是每个域的变量
As well as changing the solver, you may wish to change the number of points in the mesh. Let's look at the default number of points, which are stored as a dictionary whose keys are the variables for each domain

In [6]:
model.default_var_pts

{'x_n': 20,
 'x_s': 20,
 'x_p': 20,
 'r_n': 20,
 'r_p': 20,
 'r_n_prim': 20,
 'r_p_prim': 20,
 'r_n_sec': 20,
 'r_p_sec': 20,
 'y': 10,
 'z': 10,
 'R_n': 30,
 'R_p': 30}

To run a simulation with a different number of points we can define our own dictionary. To do this we first need to get the spatial variables used in pybamm

In [10]:
var = pybamm.standard_spatial_vars  

We can then get the spatial variable in each domain, e.g. in the negative electrode we use `x_n`

In [11]:
var.x_n

SpatialVariable(-0x49cd92fe0cb7780d, x_n, children=[], domains={'primary': ['negative electrode'], 'secondary': ['current collector']})

We might want to investigate the influence of the mesh on the solution. We first create a list of the number of points per domain we would like to use

In [12]:
npts = [16, 8, 4]

and then loop over the list, creating and solving simulations as we go. The solutions are stored in the list `sols`. Here we use the parameters from a paper by Ecker et al. from 2015.

In [13]:
params = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Ecker2015)

sols = []
var = pybamm.standard_spatial_vars 
for N in npts:
    var_pts = {
        var.x_n: N,
        var.x_s: N,
        var.x_p: N,
        var.r_n: N,
        var.r_p: N,
    }
    sim = pybamm.Simulation(model, solver=solver, parameter_values=params, var_pts=var_pts, C_rate=2)
    sol = sim.solve([0, 1800])
    sols.append(sol)

We can then plot the solutions to see how the number of points used in the discretisation affects the solution

In [14]:
pybamm.dynamic_plot(sols, ["X-averaged negative particle concentration", "Terminal voltage [V]"], labels=npts)

interactive(children=(FloatSlider(value=0.0, description='t', max=1800.0, step=18.0), Output()), _dom_classes=…

It is also possible to change other aspects of the solution procedure, such as the type of mesh or the spatial method, but we do not cover that here.